In [1]:
import tensorflow as tf
from read_data import dataset2
import os
import numpy as np
import random

In [2]:
#定义存储地址与名称
CNN = '/home/aa/CaptchaRecognition/data/cnn'
cnn = 'Captcha'
#定义模型文件所在的文件夹，若不存在则自动创建
ckpt = tf.train.latest_checkpoint(CNN)
if not ckpt:
    if not os.path.exists(CNN):
        os.mkdir(CNN)

In [3]:
# 卷积神经网络（有正则化的添加与使用）
def weight_variable(shape):  
    # 使用截断正态分布生成卷积核
    initial = tf.truncated_normal(shape, stddev=0.1)  
    return tf.Variable(initial)  
  

def bias_variable(shape):  
    # 使用relu激活函数，用一个正偏置值较准
    initial = tf.constant(0.1, shape=shape)  
    return tf.Variable(initial)  
  

def conv2d(x, W, strides=[1, 1, 1, 1]):  
    # 定义卷积层
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')   


# min_next_batch_tfr(随机批次载入数据)
def min_next_batch_tfr(image, label, num=50, num1=500): 
    images = np.zeros((num, 2240))
    labels = np.zeros((num, 44))
    for i in range(num):
        temp = random.randint(0, num1-1)
        images[i, :] = image[temp]
        labels[i, :] = label[temp]

    return images, labels


x = tf.placeholder(tf.float32, [None, 2240])
y_1 = tf.placeholder(tf.float32, [None, 11])
y_2 = tf.placeholder(tf.float32, [None, 11])
y_3 = tf.placeholder(tf.float32, [None, 11])
y_4 = tf.placeholder(tf.float32, [None, 11])

with tf.variable_scope('model') as scope:
    # 格式转换
    x_image = tf.reshape(x, [-1, 40, 56, 1])
    
    # conv1，卷积核尺寸为2*2, 通道数为1，输出通道为32
    with tf.variable_scope('conv1') as scope:
        W_conv1 = weight_variable([2, 2, 1, 32])  
        b_conv1 = bias_variable([32])  
        h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1) 
        
    # conv2，卷积核尺寸为2*2, 通道数为32，输出通道为48
    with tf.variable_scope('conv2') as scope:
        W_conv2 = weight_variable([2, 2, 32, 48])  
        b_conv2 = bias_variable([48])  
        h_conv2 = tf.nn.relu(conv2d(h_conv1, W_conv2, [1, 2, 2, 1]) + b_conv2) 

    # conv3，卷积核尺寸为2*2, 输入通道为48，输出通道为64
    with tf.variable_scope('conv3') as scope:
        W_conv3 = weight_variable([2, 2, 48, 64])  
        b_conv3 = bias_variable([64])  
        h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3) + b_conv3)  

    # conv4，卷积核尺寸为2*2, 通道数为64，输出通道为72
    with tf.variable_scope('conv4') as scope:
        W_conv4 = weight_variable([2, 2, 64, 72])  
        b_conv4 = bias_variable([72])  
        h_conv4 = tf.nn.relu(conv2d(h_conv3, W_conv4, [1, 2, 2, 1]) + b_conv4) 

    # conv5，卷积核尺寸为2*2, 输入通道为72，输出通道为96
    with tf.variable_scope('conv5') as scope:
        W_conv5 = weight_variable([2, 2, 72, 96])  
        b_conv5 = bias_variable([96])  
        h_conv5 = tf.nn.relu(conv2d(h_conv4, W_conv5) + b_conv5)
        
    # conv6，卷积核尺寸为2*2, 通道数为96，输出通道为128
    with tf.variable_scope('conv6') as scope:
        W_conv6 = weight_variable([2, 2, 96, 128])  
        b_conv6 = bias_variable([128])  
        h_conv6 = tf.nn.relu(conv2d(h_conv5, W_conv6, [1, 2, 2, 1]) + b_conv6)  
        
    # conv7，卷积核尺寸为2*2, 通道数为128，输出通道为256
    with tf.variable_scope('conv7') as scope:
        W_conv7 = weight_variable([2, 2, 128, 256])  
        b_conv7 = bias_variable([256])  
        h_conv7 = tf.nn.relu(conv2d(h_conv6, W_conv7) + b_conv7)  
        
    # conv8，卷积核尺寸为2*2, 通道数为256，输出通道为256
    with tf.variable_scope('conv8') as scope:
        W_conv8 = weight_variable([2, 2, 256, 256])  
        b_conv8 = bias_variable([256])  
        h_conv8 = tf.nn.relu(conv2d(h_conv7, W_conv8) + b_conv8)  
        
    # conv9，卷积核尺寸为2*2, 通道数为256，输出通道为512
    with tf.variable_scope('conv9') as scope:
        W_conv9 = weight_variable([2, 2, 256, 512])  
        b_conv9 = bias_variable([512])  
        h_conv9 = tf.nn.relu(conv2d(h_conv8, W_conv9) + b_conv9)  
        
    # conv10，卷积核尺寸为2*2, 通道数为512，输出通道为256
    with tf.variable_scope('conv10') as scope:
        W_conv10 = weight_variable([2, 2, 512, 256])  
        b_conv10 = bias_variable([256])  
        h_conv10 = tf.nn.relu(conv2d(h_conv9, W_conv10) + b_conv10)  
        
    # pool,转化为1*1
    with tf.variable_scope('pool') as scope:
        pool = tf.nn.avg_pool(
            h_conv10, 
            [1, 5, 7, 1], 
            [1, 5, 7, 1], 
            padding='VALID')
        flatten = tf.reshape(pool, shape=[-1, 256])
        
    # fc21,输入256维，输出11维，预测标签第一位
    with tf.variable_scope('fc21') as scope:
        W_fc21 = weight_variable([256,11])
        b_fc21 = bias_variable([11])   
        y_conv1 =tf.matmul(flatten, W_fc21) + b_fc21
    # fc22,输入256维，输出11维，预测标签第二位
    with tf.variable_scope('fc22') as scope:
        W_fc22 = weight_variable([256,11])
        b_fc22 = bias_variable([11])   
        y_conv2 = tf.matmul(flatten, W_fc22) + b_fc22
    # fc23,输入256维，输出11维，预测标签第三位
    with tf.variable_scope('fc23') as scope:
        W_fc23 = weight_variable([256,11])
        b_fc23 = bias_variable([11])
        y_conv3 = tf.matmul(flatten, W_fc23) + b_fc23
    # fc24,输入256维，输出11维，预测标签第四位
    with tf.variable_scope('fc24') as scope:
        W_fc24 = weight_variable([256,11])
        b_fc24 = bias_variable([11])   
        y_conv4 = tf.matmul(flatten, W_fc24) + b_fc24


with tf.variable_scope('calculation') as scope:
    # 损失函数，交叉熵
    cross_entropy11 = tf.nn.softmax_cross_entropy_with_logits(labels=y_1, logits=y_conv1)
    cross_entropy22 = tf.nn.softmax_cross_entropy_with_logits(labels=y_2, logits=y_conv2)
    cross_entropy33 = tf.nn.softmax_cross_entropy_with_logits(labels=y_3, logits=y_conv3)
    cross_entropy44 = tf.nn.softmax_cross_entropy_with_logits(labels=y_4, logits=y_conv4)
    cross_entropy1 = tf.reduce_mean(cross_entropy11)
    cross_entropy2 = tf.reduce_mean(cross_entropy22)
    cross_entropy3 = tf.reduce_mean(cross_entropy33)
    cross_entropy4 = tf.reduce_mean(cross_entropy44)
    cross_entropy = cross_entropy1 + cross_entropy2 + cross_entropy3 + cross_entropy4
    
    # 使用adam优化
    train_step = tf.train.AdamOptimizer().minimize(cross_entropy)

    # 正确率计算
    correct_prediction1 = tf.equal(tf.argmax(y_conv1, 1), tf.argmax(y_1, 1))  
    accuracy1 = tf.reduce_mean(tf.cast(correct_prediction1, tf.float32))
    correct_prediction2 = tf.equal(tf.argmax(y_conv2, 1), tf.argmax(y_2, 1))
    accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
    correct_prediction3 = tf.equal(tf.argmax(y_conv3, 1), tf.argmax(y_3, 1))
    accuracy3 = tf.reduce_mean(tf.cast(correct_prediction3, tf.float32))
    correct_prediction4 = tf.equal(tf.argmax(y_conv4, 1), tf.argmax(y_4, 1))
    accuracy4 = tf.reduce_mean(tf.cast(correct_prediction4, tf.float32))
    
    acce = tf.reduce_mean([accuracy1, accuracy2, accuracy3, accuracy4])
    corr = tf.reduce_all([correct_prediction1, correct_prediction2, correct_prediction3, correct_prediction4], 0)
    acc = tf.reduce_mean(tf.cast(corr, tf.float32))
    
    # 提取摘要
    tf.summary.scalar('cross_entropy', cross_entropy)
    tf.summary.scalar('accuracy', acc)
    merged = tf.summary.merge_all()

saver = tf.train.Saver(max_to_keep=1)

In [4]:
# 打印模型参数数量
from functools import reduce
from operator import mul

def get_num_params():
    num_params = 0
    for variable in tf.trainable_variables():
        shape = variable.get_shape()
        num_params += reduce(mul, [dim.value for dim in shape], 1)
    return num_params

print('参数数量为：%d' %get_num_params())

参数数量为：1568612


In [5]:
# 加载数据
with tf.variable_scope('load_data') as scope:
    train_i = np.empty((4720*4, 2240))
    train_l = np.empty((4720*4, 44))
    for w in range(4):
        train_i[0+4720*w:4720+4720*w], train_l[0+4720*w:4720+4720*w] = dataset2(
            ('train%d' %(w+1)),4720)
    validation_i,validation_l = dataset2('validation1',2360)
    test_i,test_l = dataset2('test1',2360)

training...
done
training...
done
training...
done
training...
done
training...
done
training...
done


In [6]:
# 训练模型，训练过程中每一百次插播验证集当前识别率
with tf.Session() as sess: 
    # 运行会话
    sess.run(tf.global_variables_initializer())
    # 变量定义
    step = 0
    losslist = []
    minloss = 10000
    # 若模型存在，自动加载模型进会话
    ckpt = tf.train.latest_checkpoint(CNN)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
    ckptname=os.path.join(CNN, cnn)
    writer1 = tf.summary.FileWriter('./train_graphs', filename_suffix='.file')
    writer2 = tf.summary.FileWriter('./validation_graphs', filename_suffix='.file')
    writer1.add_graph(tf.get_default_graph())
    # 开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    # 训练
    for i in range(45000):
        im, l = min_next_batch_tfr(train_i, train_l, 150, 4720*4)
        l1=l[:,0:11]
        l2=l[:,11:22]
        l3=l[:,22:33]
        l4=l[:,33:44]
        loss_, _=sess.run([cross_entropy, train_step], feed_dict={
                              x: im, y_1: l1,y_2: l2, y_3: l3, y_4: l4
                          })
        # 每训练3000个用验证集测试150个
        if i%20 ==0:
            # 训练代价摘要
            m1 = sess.run(merged, feed_dict={x: im, y_1: l1,y_2: l2, y_3: l3, y_4: l4})
            writer1.add_summary(m1, global_step=i)
            v_i, v_l = min_next_batch_tfr(validation_i,validation_l, 1000, 2360)
            v_l1=v_l[:,0:11]
            v_l2=v_l[:,11:22]
            v_l3=v_l[:,22:33]
            v_l4=v_l[:,33:44]
            m2, accev, accu, loss = sess.run([merged, acce, acc, cross_entropy], 
                feed_dict={
                    x: v_i, y_1: v_l1, y_2: v_l2, y_3: v_l3, y_4: v_l4
                })
            writer2.add_summary(m2, global_step=i)
            print("step %d, train loss is %f, validation loss %f" % (i//20, loss_, loss))
            print("验证码字符识别正确率 %f" % accev)
            print("验证码识别正确率 %f" % accu)
            print("平均正确率 %f" % np.mean([accu,accev]))
            print("-----------------------------------------------------------")
            losslist.append(loss)
        # 保存损失最低的模型
        if minloss > loss:
            minloss = loss
            saver.save(sess,ckptname,global_step=150*i)
        if losslist[-1] > minloss and losslist[-2] > minloss and losslist[-3] > minloss and losslist[-4] > minloss:
            break

    coord.request_stop() 
    coord.join(threads)
    writer1.close()
    writer2.close()

step 0, train loss is 34.881283, validation loss 19.219315
验证码字符识别正确率 0.241750
验证码识别正确率 0.000000
平均正确率 0.120875
-----------------------------------------------------------
step 1, train loss is 7.470591, validation loss 7.490627
验证码字符识别正确率 0.409250
验证码识别正确率 0.025000
平均正确率 0.217125
-----------------------------------------------------------
step 2, train loss is 7.547095, validation loss 7.452349
验证码字符识别正确率 0.420750
验证码识别正确率 0.038000
平均正确率 0.229375
-----------------------------------------------------------
step 3, train loss is 7.432042, validation loss 7.562891
验证码字符识别正确率 0.397000
验证码识别正确率 0.028000
平均正确率 0.212500
-----------------------------------------------------------
step 4, train loss is 7.585989, validation loss 7.366376
验证码字符识别正确率 0.419250
验证码识别正确率 0.037000
平均正确率 0.228125
-----------------------------------------------------------
step 5, train loss is 7.527457, validation loss 7.417470
验证码字符识别正确率 0.407000
验证码识别正确率 0.030000
平均正确率 0.218500
--------------------------------------

In [6]:
# 模型在各个数据集（训练集、验证集、测试集）上正确率的期望
with tf.Session() as sess: 
    # 运行会话
    sess.run(tf.global_variables_initializer())
    # 加载模型
    ckpt = tf.train.latest_checkpoint(CNN)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
    ckptname=os.path.join(CNN, cnn)
    # 开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # 训练集
    im, tr_l = min_next_batch_tfr(train_i, train_l, 5000, 4720*4)
    tr_l1 = tr_l[:,0:11]
    tr_l2 = tr_l[:,11:22]
    tr_l3 = tr_l[:,22:33]
    tr_l4 = tr_l[:,33:44]
    traccu = sess.run(acc, 
        feed_dict={
            x: im, y_1: tr_l1, y_2: tr_l2, y_3: tr_l3, y_4: tr_l4
        })
    # 验证集
    va_l1 = validation_l[:,0:11]
    va_l2 = validation_l[:,11:22]
    va_l3 = validation_l[:,22:33]
    va_l4 = validation_l[:,33:44]
    vaccu = sess.run(acc, 
        feed_dict={
            x: validation_i, y_1: va_l1, y_2: va_l2, y_3: va_l3, y_4: va_l4
        })
    # 测试集
    t_l1 = test_l[:,0:11]
    t_l2 = test_l[:,11:22]
    t_l3 = test_l[:,22:33]
    t_l4 = test_l[:,33:44]
    teaccu = sess.run(acc, 
        feed_dict={
            x: test_i, y_1: t_l1, y_2: t_l2, y_3: t_l3, y_4: t_l4
        })
    print("训练集验证码识别正确率 %f" % traccu)
    print("验证集验证码识别正确率 %f" % vaccu)
    print("测试集验证码识别正确率 %f" % teaccu)

    coord.request_stop() 
    coord.join(threads)

INFO:tensorflow:Restoring parameters from /home/aa/CaptchaRecognition/data/cnn/Captcha-168000
训练集验证码识别正确率 0.743400
验证集验证码识别正确率 0.605508
测试集验证码识别正确率 0.590678


In [6]:
# 测试集测试泛化能力
with tf.Session() as sess: 
    # 运行会话
    sess.run(tf.global_variables_initializer())
    # 加载模型
    ckpt = tf.train.latest_checkpoint(CNN)
    if ckpt:
        saver.restore(sess=sess,save_path=ckpt)
        step = int(ckpt[len(os.path.join(CNN, cnn)) + 1:])
    ckptname=os.path.join(CNN, cnn)
    # 开启线程
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    # 测试
    test_i,test_l
    t_l1 = test_l[:,0:11]
    t_l2 = test_l[:,11:22]
    t_l3 = test_l[:,22:33]
    t_l4 = test_l[:,33:44]
    accev, accu = sess.run([acce, acc], 
        feed_dict={
            x: test_i, y_1: t_l1, y_2: t_l2, y_3: t_l3, y_4: t_l4
        })
    print("测试集验证码字符识别正确率 %f" % accev)
    print("测试集验证码识别正确率 %f" % accu)
    print("测试集平均正确率 %f" % np.mean([accu,accev]))

    coord.request_stop() 
    coord.join(threads)

INFO:tensorflow:Restoring parameters from /home/aa/CaptchaRecognition/data/cnn/Captcha-168000
测试集验证码字符识别正确率 0.856674
测试集验证码识别正确率 0.590678
测试集平均正确率 0.723676
